在tensorflow中，提供了tf.nn.conv2d()的函数，用于卷积运算，这适合用于底层的运算。此外，tensorflow还提供了layers.Conv2D的卷积层实现，可以自动的处理各种问题。

通过卷积层类layers.Conv2D可以不需要手动定义卷积核W和偏置b张量，直接调用类实例即可完成卷积层的前向计算，实现更加高层和快捷。在TensorFlow中，API的命名有一定的规律，首字母大写的对象一般表示类，全部小写的一般表示函数，如layers.Conv2D表示卷积层类，nn.conv2d表示卷积运算函数。使用类方式会（在创建类时或build时）自动创建需要的权值张量和偏置向量等，用户不需要记忆卷积核张量的定义格式，因此使用起来更简单方便，但是灵活性也略低。函数方式的接口需要自行定义权值和偏置等，更加灵活和底层。

下面我们分别介绍一下。

In [1]:
import tensorflow as tf
from tensorflow import keras

# 卷积运算函数

我们举一个例子，我们随机生产2个矩阵，然后让他们进行矩阵运算：

In [8]:
f = tf.random.normal([10,28,28,3]) # 可以理解为10张大小为28*28的图片，通道数量为3.
g = tf.random.normal([5,5,3,25]) # 滤波器大小为5*5，通道数量为3（必须与输入相同），25个滤波器。

result = tf.nn.conv2d(f,g,strides=1,padding=([0,0],[0,0],[0,0],[0,0]))
print(result.shape)

TensorShape([10, 24, 24, 25])

简单说明一下：
* strides表示步长。
* padding表示填充，第1、4个参数暂时没有用，2、3个参数表示上下左右分别填充什么数值，一般为0。

# 卷积层
在使用keras等high level API构建网络结构时，我们一般会使用tf.layers.Conv2D，比如：

In [10]:
layer = keras.layers.Conv2D(filters=4, kernel_size=3,strides=1, padding='SAME')

我们对上述的参数做一些解释：

* images是输入的小批量（4D张量，如前所述）。
* filter是要应用的一组滤波器（也是4D张量，如前所述）。
* strides等于1，但也可以是包含四个元素的一维度组，其中两个中间元素是垂直步幅和水平步幅（sh和sw）。第一个元素和最后一个元素必须等于1。它们可能有一天用于指定批处理步幅（跳过某些实例）和通道步幅（跳过某些上一层的特征图或通道）。
* padding必须为"SAME"或"VALID"：
* 如果设置为"SAME"，则卷积层在必要时使用零填充。将输出大小设置为输入神经元的数量除以步幅（向上取整）所得的值。例如，如果输入大小为13而步幅为5，则输出大小为3（即13/5=2.6，向上舍入为3）。然后根据需要在输入周围尽可能均匀地添加零。当strides=1时，层的输出将具有与其输入相同的空间尺寸（宽度和高度），因此命名为“same”。
* 如果设置为"VALID"，则卷积层将不使用零填充，并且可能会忽略输入图像底部和右侧的某些行和列，具体取决于步幅，（为简单起见，仅这里显示了水平尺寸，当然垂直尺寸也适用相同的逻辑。这意味着每个神经元的接受野都严格位于输入内部的有效位置内（不会超出范围），因此命名为“valid”。